# Multi-Task Drug Response Prediction

**Goal**: Predict IC50 values for multiple drugs simultaneously

**Why Multi-Task?**
- Single-drug prediction can overfit to drug-specific patterns
- Multi-task forces the model to learn generalizable biological features
- More realistic evaluation: can the model predict drug responses broadly?

**Problem Difficulty Options**:
1. **Random split** (Baseline): Standard 80/20 random split
2. **Histology-based split** (Hard): Test on unseen cancer subtypes
3. **Site-based split** (Hard): Test on unseen tissue origins

## Google Colab Setup
Run the cell below first to set up the environment.

In [ ]:
# Google Colab Setup - Run this cell first!
import os

IN_COLAB = 'google.colab' in str(get_ipython()) if 'get_ipython' in dir() else False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_DIR = '/content/drive/MyDrive/bioai_data'
    os.chdir(DATA_DIR)
    print(f"Working directory: {os.getcwd()}")
    print(f"Files available: {os.listdir('.')}")
else:
    print("Not running in Colab - using local paths")

## Data Loading

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load multi-omics feature data
methexpr_df = pd.read_csv('ml_with_gene_expr.csv.gz',
                          compression='gzip',
                          index_col=0,
                          low_memory=False)

# Separate feature types
metadata_cols = ['primary site', 'primary histology', 'cosmic_id']
methylation_cols = [col for col in methexpr_df.columns if col.startswith('cg')]
expression_cols = [col for col in methexpr_df.columns if col.startswith('expr_')]

print(f"Feature data shape: {methexpr_df.shape}")
print(f"Methylation features: {len(methylation_cols)}")
print(f"Expression features: {len(expression_cols)}")

In [ ]:
# Load drug response data
response_df = pd.read_csv('ML_dataset_methylation_drug_response.csv.gz',
                          compression='gzip',
                          index_col=0,
                          low_memory=False)

# Extract only drug columns (not methylation features or metadata)
drug_cols = [col for col in response_df.columns 
             if not col.startswith('cg') and col not in metadata_cols]

print(f"Drug response data shape: {response_df.shape}")
print(f"Total drugs available: {len(drug_cols)}")

## Drug Selection by Coverage

Select drugs with high coverage (most cell lines have IC50 values) to minimize missing data issues.

In [ ]:
# Analyze drug coverage
drug_response = response_df[drug_cols]
coverage = drug_response.notna().sum().sort_values(ascending=False)
total_samples = len(response_df)

print(f"Drug coverage statistics (of {total_samples} cell lines):")
print(f"  Max coverage: {coverage.max()} ({100*coverage.max()/total_samples:.1f}%)")
print(f"  Min coverage: {coverage.min()} ({100*coverage.min()/total_samples:.1f}%)")
print(f"  Median coverage: {coverage.median():.0f} ({100*coverage.median()/total_samples:.1f}%)")

print(f"\nDrugs by coverage threshold:")
for thresh in [0.95, 0.90, 0.85, 0.80]:
    n_drugs = (coverage > thresh * total_samples).sum()
    print(f"  >{thresh*100:.0f}% coverage: {n_drugs} drugs")

In [ ]:
# Select drugs with high coverage
# CONFIGURABLE: Adjust threshold to balance number of drugs vs missing data
COVERAGE_THRESHOLD = 0.90  # 90% coverage = 288 drugs

min_samples = int(COVERAGE_THRESHOLD * total_samples)
selected_drugs = coverage[coverage >= min_samples].index.tolist()

print(f"Coverage threshold: {COVERAGE_THRESHOLD*100:.0f}% ({min_samples} samples minimum)")
print(f"Selected drugs: {len(selected_drugs)}")
print(f"\nTop 10 drugs by coverage:")
for drug in selected_drugs[:10]:
    print(f"  {drug}: {coverage[drug]} ({100*coverage[drug]/total_samples:.1f}%)")

## Merge Features and Drug Responses

In [ ]:
# Merge feature data with drug responses (inner join on cell line names)
df = methexpr_df.join(response_df[selected_drugs], how='inner')

print(f"Merged dataset shape: {df.shape}")
print(f"Cell lines with both features and drug data: {len(df)}")

In [ ]:
# Prepare multi-task target matrix (Y)
Y_all = df[selected_drugs]

# Find cell lines with complete drug response data (no missing values)
complete_mask = Y_all.notna().all(axis=1)
n_complete = complete_mask.sum()

print(f"Cell lines with complete drug data: {n_complete} ({100*n_complete/len(df):.1f}%)")

# Alternative: use cell lines with >X% drug coverage
CELL_COVERAGE_THRESHOLD = 0.95
cell_coverage = Y_all.notna().sum(axis=1) / len(selected_drugs)
usable_cells = cell_coverage >= CELL_COVERAGE_THRESHOLD

print(f"Cell lines with >{CELL_COVERAGE_THRESHOLD*100:.0f}% drug coverage: {usable_cells.sum()}")

In [ ]:
# Use cell lines with high drug coverage, impute remaining missing values
df_filtered = df[usable_cells].copy()

# Prepare features (X) and targets (Y)
feature_cols = methylation_cols + expression_cols
X = df_filtered[feature_cols]

# Drop columns with any NaN (matches harder_ds.ipynb approach)
# This removes ~336 methylation probes with technical failures
X = X.dropna(axis=1)

Y = df_filtered[selected_drugs]

# Impute remaining missing drug values with drug-specific median
Y_imputed = Y.fillna(Y.median())

print(f"Final dataset:")
print(f"  X (features): {X.shape}")
print(f"  Y (drug responses): {Y_imputed.shape}")
print(f"  Dropped {len(feature_cols) - X.shape[1]} features with NaN (matching harder_ds.ipynb)")
print(f"  Missing values in Y after imputation: {Y_imputed.isna().sum().sum()}")

## Multi-Task Evaluation Functions

In [ ]:
def evaluate_multitask(model, X_train, Y_train, X_test, Y_test):
    """
    Evaluate multi-output regression model.
    
    Returns:
        dict with per-drug and aggregate metrics
    """
    # Predictions
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)
    
    # Convert to DataFrame for easier analysis
    Y_pred_train_df = pd.DataFrame(Y_pred_train, columns=Y_train.columns, index=Y_train.index)
    Y_pred_test_df = pd.DataFrame(Y_pred_test, columns=Y_test.columns, index=Y_test.index)
    
    # Per-drug metrics
    train_r2_per_drug = []
    test_r2_per_drug = []
    test_mse_per_drug = []
    
    for drug in Y_train.columns:
        train_r2_per_drug.append(r2_score(Y_train[drug], Y_pred_train_df[drug]))
        test_r2_per_drug.append(r2_score(Y_test[drug], Y_pred_test_df[drug]))
        test_mse_per_drug.append(mean_squared_error(Y_test[drug], Y_pred_test_df[drug]))
    
    # Aggregate metrics
    results = {
        'train_r2_mean': np.mean(train_r2_per_drug),
        'train_r2_median': np.median(train_r2_per_drug),
        'test_r2_mean': np.mean(test_r2_per_drug),
        'test_r2_median': np.median(test_r2_per_drug),
        'test_mse_mean': np.mean(test_mse_per_drug),
        'test_r2_positive_frac': np.mean(np.array(test_r2_per_drug) > 0),
        'per_drug_test_r2': dict(zip(Y_train.columns, test_r2_per_drug)),
        'per_drug_test_mse': dict(zip(Y_train.columns, test_mse_per_drug)),
    }
    
    return results

def print_multitask_results(results, model_name):
    """Pretty print multi-task evaluation results."""
    print(f"\n{'='*60}")
    print(f"Model: {model_name}")
    print(f"{'='*60}")
    print(f"Training R² (mean across drugs): {results['train_r2_mean']:.4f}")
    print(f"Training R² (median): {results['train_r2_median']:.4f}")
    print(f"\nTest R² (mean across drugs): {results['test_r2_mean']:.4f}")
    print(f"Test R² (median): {results['test_r2_median']:.4f}")
    print(f"Test MSE (mean): {results['test_mse_mean']:.4f}")
    print(f"\nDrugs with positive test R²: {results['test_r2_positive_frac']*100:.1f}%")
    
    # Show best and worst predicted drugs
    sorted_drugs = sorted(results['per_drug_test_r2'].items(), key=lambda x: x[1], reverse=True)
    print(f"\nTop 5 best predicted drugs:")
    for drug, r2 in sorted_drugs[:5]:
        print(f"  {drug}: R²={r2:.4f}")
    print(f"\nTop 5 worst predicted drugs:")
    for drug, r2 in sorted_drugs[-5:]:
        print(f"  {drug}: R²={r2:.4f}")

---
# Split Strategy 0: Random Split (Baseline)

**Goal**: Establish baseline performance with standard random train/test split.

This is the "easy" setting because:
- Test samples are randomly selected, likely similar to training samples
- Same cancer types and tissues appear in both train and test
- Model can leverage superficial patterns that don't generalize

In [ ]:
from sklearn.model_selection import train_test_split

# Standard 80/20 random split
TEST_SIZE = 0.20
X_train_rand, X_test_rand, Y_train_rand, Y_test_rand = train_test_split(
    X, Y_imputed, test_size=TEST_SIZE, random_state=42
)

print(f"Random split ({(1-TEST_SIZE)*100:.0f}/{TEST_SIZE*100:.0f}):")
print(f"  Train samples: {len(X_train_rand)} ({100*len(X_train_rand)/len(X):.1f}%)")
print(f"  Test samples: {len(X_test_rand)} ({100*len(X_test_rand)/len(X):.1f}%)")

In [ ]:
# Scale and PCA for random split
scaler_rand = RobustScaler()
X_train_rand_scaled = scaler_rand.fit_transform(X_train_rand)
X_test_rand_scaled = scaler_rand.transform(X_test_rand)

N_COMPONENTS = 50  # More components for multi-task (capturing more variance)
pca_rand = PCA(n_components=N_COMPONENTS)
X_train_rand_pca = pca_rand.fit_transform(X_train_rand_scaled)
X_test_rand_pca = pca_rand.transform(X_test_rand_scaled)

print(f"PCA variance explained: {pca_rand.explained_variance_ratio_.sum()*100:.1f}%")
print(f"X_train shape after PCA: {X_train_rand_pca.shape}")
print(f"X_test shape after PCA: {X_test_rand_pca.shape}")

## Model Training: Random Split (Baseline)

In [ ]:
# Initialize benchmark results for random split
benchmark_rand = pd.DataFrame(columns=[
    'Model', 'Train R² (mean)', 'Train R² (median)', 
    'Test R² (mean)', 'Test R² (median)', 'Test MSE (mean)', '% Drugs R²>0'
])

In [ ]:
from sklearn.linear_model import Ridge

# Ridge Regression (Random Split)
ridge_rand = Ridge(alpha=1.0)
ridge_rand.fit(X_train_rand_pca, Y_train_rand)

results_ridge_rand = evaluate_multitask(ridge_rand, X_train_rand_pca, Y_train_rand, X_test_rand_pca, Y_test_rand)
print_multitask_results(results_ridge_rand, 'Ridge Regression (Random Split)')

benchmark_rand.loc[len(benchmark_rand)] = [
    'Ridge', results_ridge_rand['train_r2_mean'], results_ridge_rand['train_r2_median'],
    results_ridge_rand['test_r2_mean'], results_ridge_rand['test_r2_median'],
    results_ridge_rand['test_mse_mean'], results_ridge_rand['test_r2_positive_frac']*100
]

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest (Random Split)
rf_rand = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_rand.fit(X_train_rand_pca, Y_train_rand)

results_rf_rand = evaluate_multitask(rf_rand, X_train_rand_pca, Y_train_rand, X_test_rand_pca, Y_test_rand)
print_multitask_results(results_rf_rand, 'Random Forest (Random Split)')

benchmark_rand.loc[len(benchmark_rand)] = [
    'Random Forest', results_rf_rand['train_r2_mean'], results_rf_rand['train_r2_median'],
    results_rf_rand['test_r2_mean'], results_rf_rand['test_r2_median'],
    results_rf_rand['test_mse_mean'], results_rf_rand['test_r2_positive_frac']*100
]

In [ ]:
from xgboost import XGBRegressor

# XGBoost (Random Split)
xgb_base_rand = XGBRegressor(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.1,
    random_state=42, 
    tree_method='hist', 
    device='cuda',
    verbosity=0
)
xgb_multi_rand = MultiOutputRegressor(xgb_base_rand, n_jobs=1)

print(f"Training XGBoost for {len(selected_drugs)} drugs (GPU-accelerated)...")
xgb_multi_rand.fit(X_train_rand_pca, Y_train_rand)

results_xgb_rand = evaluate_multitask(xgb_multi_rand, X_train_rand_pca, Y_train_rand, X_test_rand_pca, Y_test_rand)
print_multitask_results(results_xgb_rand, 'XGBoost (Random Split)')

benchmark_rand.loc[len(benchmark_rand)] = [
    'XGBoost', results_xgb_rand['train_r2_mean'], results_xgb_rand['train_r2_median'],
    results_xgb_rand['test_r2_mean'], results_xgb_rand['test_r2_median'],
    results_xgb_rand['test_mse_mean'], results_xgb_rand['test_r2_positive_frac']*100
]

In [ ]:
print("\n" + "="*70)
print("BENCHMARK RESULTS: RANDOM SPLIT (Baseline)")
print(f"Train: {len(X_train_rand)} samples | Test: {len(X_test_rand)} samples")
print(f"Drugs: {len(selected_drugs)} | PCA components: {N_COMPONENTS}")
print("="*70)
display(benchmark_rand)

---
# Split Strategy 1: Histology-Based (Unseen Cancer Subtypes)

**Goal**: Test model's ability to predict drug response for cancer subtypes never seen during training.

This is a challenging test because:
- Different cancer subtypes have distinct molecular profiles
- A rare cancer like chondrosarcoma may have completely different drug sensitivities than common cancers like lung adenocarcinoma

In [ ]:
# Histology distribution
histology = df_filtered['primary histology']
hist_counts = histology.value_counts(ascending=True)

print(f"Histology types: {len(hist_counts)}")
print(f"\nDistribution (ascending by count):")
print(hist_counts)

In [ ]:
# Split by histology: rare types go to test
HISTOLOGY_SPLIT_THRESHOLD = 0.40  # 40% of histology types go to test

split_idx = int(len(hist_counts) * HISTOLOGY_SPLIT_THRESHOLD)
test_histologies = hist_counts.index[:split_idx].tolist()  # Rarest types
train_histologies = hist_counts.index[split_idx:].tolist()  # Common types

print(f"Histology split at {HISTOLOGY_SPLIT_THRESHOLD*100:.0f}% threshold:")
print(f"  Test histologies: {len(test_histologies)} types (rare, unseen)")
print(f"  Train histologies: {len(train_histologies)} types (common)")

# Create train/test masks
train_mask_hist = histology.isin(train_histologies)
test_mask_hist = histology.isin(test_histologies)

X_train_hist = X[train_mask_hist]
X_test_hist = X[test_mask_hist]
Y_train_hist = Y_imputed[train_mask_hist]
Y_test_hist = Y_imputed[test_mask_hist]

print(f"\nTrain samples: {len(X_train_hist)} ({100*len(X_train_hist)/len(X):.1f}%)")
print(f"Test samples: {len(X_test_hist)} ({100*len(X_test_hist)/len(X):.1f}%)")
print(f"\nTest histology types: {test_histologies[:10]}..." if len(test_histologies) > 10 else f"\nTest histology types: {test_histologies}")

## Feature Scaling and Dimensionality Reduction

**Critical**: Methylation (0-1) and expression (2-14) have different scales. Must scale before PCA.

In [ ]:
# Scale features (critical: methylation 0-1, expression 2-14)
scaler_hist = RobustScaler()
X_train_hist_scaled = scaler_hist.fit_transform(X_train_hist)
X_test_hist_scaled = scaler_hist.transform(X_test_hist)

# Dimensionality reduction with PCA (using same N_COMPONENTS as random split)
pca_hist = PCA(n_components=N_COMPONENTS)
X_train_hist_pca = pca_hist.fit_transform(X_train_hist_scaled)
X_test_hist_pca = pca_hist.transform(X_test_hist_scaled)

print(f"PCA variance explained: {pca_hist.explained_variance_ratio_.sum()*100:.1f}%")
print(f"X_train shape after PCA: {X_train_hist_pca.shape}")
print(f"X_test shape after PCA: {X_test_hist_pca.shape}")

## Model Training: Histology-Based Split

In [ ]:
# Initialize benchmark results
benchmark_hist = pd.DataFrame(columns=[
    'Model', 'Train R² (mean)', 'Train R² (median)', 
    'Test R² (mean)', 'Test R² (median)', 'Test MSE (mean)', '% Drugs R²>0'
])

### Ridge Regression (Multi-Output)

In [ ]:
from sklearn.linear_model import Ridge

# Ridge regression handles multi-output natively
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_hist_pca, Y_train_hist)

results_ridge = evaluate_multitask(ridge, X_train_hist_pca, Y_train_hist, X_test_hist_pca, Y_test_hist)
print_multitask_results(results_ridge, 'Ridge Regression')

benchmark_hist.loc[len(benchmark_hist)] = [
    'Ridge', results_ridge['train_r2_mean'], results_ridge['train_r2_median'],
    results_ridge['test_r2_mean'], results_ridge['test_r2_median'],
    results_ridge['test_mse_mean'], results_ridge['test_r2_positive_frac']*100
]

### Random Forest (Multi-Output)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# RandomForest handles multi-output natively
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train_hist_pca, Y_train_hist)

results_rf = evaluate_multitask(rf, X_train_hist_pca, Y_train_hist, X_test_hist_pca, Y_test_hist)
print_multitask_results(results_rf, 'Random Forest')

benchmark_hist.loc[len(benchmark_hist)] = [
    'Random Forest', results_rf['train_r2_mean'], results_rf['train_r2_median'],
    results_rf['test_r2_mean'], results_rf['test_r2_median'],
    results_rf['test_mse_mean'], results_rf['test_r2_positive_frac']*100
]

### XGBoost (Multi-Output via MultiOutputRegressor)

In [ ]:
from xgboost import XGBRegressor

# XGBoost needs MultiOutputRegressor wrapper for multi-output
# Note: This trains a separate XGBoost per drug - computationally intensive
xgb_base = XGBRegressor(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.1,
    random_state=42, 
    tree_method='hist', 
    device='cuda',
    verbosity=0
)
xgb_multi = MultiOutputRegressor(xgb_base, n_jobs=1)  # n_jobs=1 since GPU handles parallelism

print(f"Training XGBoost for {len(selected_drugs)} drugs (GPU-accelerated)...")
xgb_multi.fit(X_train_hist_pca, Y_train_hist)

results_xgb = evaluate_multitask(xgb_multi, X_train_hist_pca, Y_train_hist, X_test_hist_pca, Y_test_hist)
print_multitask_results(results_xgb, 'XGBoost (Multi-Output)')

benchmark_hist.loc[len(benchmark_hist)] = [
    'XGBoost', results_xgb['train_r2_mean'], results_xgb['train_r2_median'],
    results_xgb['test_r2_mean'], results_xgb['test_r2_median'],
    results_xgb['test_mse_mean'], results_xgb['test_r2_positive_frac']*100
]

In [ ]:
print("\n" + "="*70)
print("BENCHMARK RESULTS: HISTOLOGY-BASED SPLIT (Unseen Cancer Subtypes)")
print(f"Train: {len(X_train_hist)} samples | Test: {len(X_test_hist)} samples")
print(f"Drugs: {len(selected_drugs)} | PCA components: {N_COMPONENTS}")
print("="*70)
display(benchmark_hist)

---
# Split Strategy 2: Primary Site-Based (Unseen Tissue Origins)

**Goal**: Test model's ability to predict drug response for tissues never seen during training.

This tests a different kind of generalization:
- Can a model trained on lung/blood cancers predict for pancreas/thyroid cancers?
- Different organs have distinct gene expression and methylation patterns

In [ ]:
# Primary site distribution
primary_site = df_filtered['primary site']
site_counts = primary_site.value_counts(ascending=True)

print(f"Primary sites: {len(site_counts)}")
print(f"\nDistribution (ascending by count):")
print(site_counts)

In [ ]:
# Split by primary site: rare tissues go to test
SITE_SPLIT_THRESHOLD = 0.40  # 40% of sites go to test

site_split_idx = int(len(site_counts) * SITE_SPLIT_THRESHOLD)
test_sites = site_counts.index[:site_split_idx].tolist()  # Rarest sites
train_sites = site_counts.index[site_split_idx:].tolist()  # Common sites

print(f"Site split at {SITE_SPLIT_THRESHOLD*100:.0f}% threshold:")
print(f"  Test sites: {len(test_sites)} ({test_sites})")
print(f"  Train sites: {len(train_sites)} ({train_sites})")

# Create train/test masks
train_mask_site = primary_site.isin(train_sites)
test_mask_site = primary_site.isin(test_sites)

X_train_site = X[train_mask_site]
X_test_site = X[test_mask_site]
Y_train_site = Y_imputed[train_mask_site]
Y_test_site = Y_imputed[test_mask_site]

print(f"\nTrain samples: {len(X_train_site)} ({100*len(X_train_site)/len(X):.1f}%)")
print(f"Test samples: {len(X_test_site)} ({100*len(X_test_site)/len(X):.1f}%)")

In [ ]:
# Scale and PCA for site-based split
scaler_site = RobustScaler()
X_train_site_scaled = scaler_site.fit_transform(X_train_site)
X_test_site_scaled = scaler_site.transform(X_test_site)

pca_site = PCA(n_components=N_COMPONENTS)
X_train_site_pca = pca_site.fit_transform(X_train_site_scaled)
X_test_site_pca = pca_site.transform(X_test_site_scaled)

print(f"PCA variance explained: {pca_site.explained_variance_ratio_.sum()*100:.1f}%")
print(f"X_train shape after PCA: {X_train_site_pca.shape}")
print(f"X_test shape after PCA: {X_test_site_pca.shape}")

## Model Training: Site-Based Split

In [ ]:
# Initialize benchmark results for site-based split
benchmark_site = pd.DataFrame(columns=[
    'Model', 'Train R² (mean)', 'Train R² (median)', 
    'Test R² (mean)', 'Test R² (median)', 'Test MSE (mean)', '% Drugs R²>0'
])

In [ ]:
# Ridge Regression
ridge_site = Ridge(alpha=1.0)
ridge_site.fit(X_train_site_pca, Y_train_site)

results_ridge_site = evaluate_multitask(ridge_site, X_train_site_pca, Y_train_site, X_test_site_pca, Y_test_site)
print_multitask_results(results_ridge_site, 'Ridge Regression (Site Split)')

benchmark_site.loc[len(benchmark_site)] = [
    'Ridge', results_ridge_site['train_r2_mean'], results_ridge_site['train_r2_median'],
    results_ridge_site['test_r2_mean'], results_ridge_site['test_r2_median'],
    results_ridge_site['test_mse_mean'], results_ridge_site['test_r2_positive_frac']*100
]

In [ ]:
# Random Forest
rf_site = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_site.fit(X_train_site_pca, Y_train_site)

results_rf_site = evaluate_multitask(rf_site, X_train_site_pca, Y_train_site, X_test_site_pca, Y_test_site)
print_multitask_results(results_rf_site, 'Random Forest (Site Split)')

benchmark_site.loc[len(benchmark_site)] = [
    'Random Forest', results_rf_site['train_r2_mean'], results_rf_site['train_r2_median'],
    results_rf_site['test_r2_mean'], results_rf_site['test_r2_median'],
    results_rf_site['test_mse_mean'], results_rf_site['test_r2_positive_frac']*100
]

In [ ]:
# XGBoost
xgb_base_site = XGBRegressor(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.1,
    random_state=42, 
    tree_method='hist', 
    device='cuda',
    verbosity=0
)
xgb_multi_site = MultiOutputRegressor(xgb_base_site, n_jobs=1)

print(f"Training XGBoost for {len(selected_drugs)} drugs (GPU-accelerated)...")
xgb_multi_site.fit(X_train_site_pca, Y_train_site)

results_xgb_site = evaluate_multitask(xgb_multi_site, X_train_site_pca, Y_train_site, X_test_site_pca, Y_test_site)
print_multitask_results(results_xgb_site, 'XGBoost (Site Split)')

benchmark_site.loc[len(benchmark_site)] = [
    'XGBoost', results_xgb_site['train_r2_mean'], results_xgb_site['train_r2_median'],
    results_xgb_site['test_r2_mean'], results_xgb_site['test_r2_median'],
    results_xgb_site['test_mse_mean'], results_xgb_site['test_r2_positive_frac']*100
]

In [ ]:
print("\n" + "="*70)
print("BENCHMARK RESULTS: SITE-BASED SPLIT (Unseen Tissue Origins)")
print(f"Train: {len(X_train_site)} samples | Test: {len(X_test_site)} samples")
print(f"Drugs: {len(selected_drugs)} | PCA components: {N_COMPONENTS}")
print("="*70)
display(benchmark_site)

---
# Final Comparison: Split Strategies

In [ ]:
print("="*80)
print("MULTI-TASK DRUG RESPONSE PREDICTION: FINAL COMPARISON")
print("="*80)

print(f"\nDataset: {len(X)} cell lines × {len(selected_drugs)} drugs")
print(f"Features: {X.shape[1]} (methylation + expression) → {N_COMPONENTS} PCA components")

print("\n" + "-"*80)
print("RANDOM SPLIT (Baseline - Easy)")
print(f"Train: {len(X_train_rand)} | Test: {len(X_test_rand)} ({100*len(X_test_rand)/len(X):.1f}%)")
print("-"*80)
display(benchmark_rand)

print("\n" + "-"*80)
print("HISTOLOGY-BASED SPLIT (Hard - Unseen Cancer Subtypes)")
print(f"Train: {len(X_train_hist)} | Test: {len(X_test_hist)} ({100*len(X_test_hist)/len(X):.1f}%)")
print("-"*80)
display(benchmark_hist)

print("\n" + "-"*80)
print("SITE-BASED SPLIT (Hard - Unseen Tissue Origins)")
print(f"Train: {len(X_train_site)} | Test: {len(X_test_site)} ({100*len(X_test_site)/len(X):.1f}%)")
print("-"*80)
display(benchmark_site)

print("\n" + "="*80)
print("INTERPRETATION")
print("="*80)
print("""
Comparing split strategies reveals model generalization capability:

1. RANDOM SPLIT (Baseline):
   - Test samples are similar to training samples
   - Higher R² expected - this is the "easy" setting
   - If R² is low even here, the task is fundamentally hard

2. HISTOLOGY SPLIT (Hard):
   - Tests generalization to UNSEEN CANCER SUBTYPES
   - Drop in R² from baseline shows subtype-specific patterns

3. SITE SPLIT (Hard):
   - Tests generalization to UNSEEN TISSUE ORIGINS
   - Drop in R² from baseline shows tissue-specific patterns

4. KEY METRIC: '% Drugs R²>0'
   - What fraction of drugs can the model predict better than baseline?
   - Compare across splits to see which drugs are robust vs tissue-specific
""")

## Per-Drug Performance Analysis

In [ ]:
# Compare per-drug R² between split strategies (using XGBoost results)
import matplotlib.pyplot as plt

# Get per-drug R² from XGBoost for all three splits
rand_r2 = pd.Series(results_xgb_rand['per_drug_test_r2'])
hist_r2 = pd.Series(results_xgb['per_drug_test_r2'])
site_r2 = pd.Series(results_xgb_site['per_drug_test_r2'])

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram of per-drug R² for all splits
axes[0].hist(rand_r2, bins=30, alpha=0.7, label='Random Split', color='green')
axes[0].hist(hist_r2, bins=30, alpha=0.7, label='Histology Split', color='blue')
axes[0].hist(site_r2, bins=30, alpha=0.7, label='Site Split', color='orange')
axes[0].axvline(x=0, color='red', linestyle='--', label='Baseline (R²=0)')
axes[0].set_xlabel('Test R² per Drug')
axes[0].set_ylabel('Number of Drugs')
axes[0].set_title('Distribution of Per-Drug Test R²')
axes[0].legend()

# Scatter: Random R² vs Histology R²
axes[1].scatter(rand_r2, hist_r2, alpha=0.5, color='blue')
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[1].plot([-0.5, 0.5], [-0.5, 0.5], 'k--', alpha=0.3, label='y=x')
axes[1].set_xlabel('Test R² (Random Split)')
axes[1].set_ylabel('Test R² (Histology Split)')
axes[1].set_title('Random vs Histology Split')
axes[1].legend()

# Scatter: Random R² vs Site R²
axes[2].scatter(rand_r2, site_r2, alpha=0.5, color='orange')
axes[2].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[2].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[2].plot([-0.5, 0.5], [-0.5, 0.5], 'k--', alpha=0.3, label='y=x')
axes[2].set_xlabel('Test R² (Random Split)')
axes[2].set_ylabel('Test R² (Site Split)')
axes[2].set_title('Random vs Site Split')
axes[2].legend()

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\nPerformance degradation from Random (baseline) to harder splits:")
print(f"  Random → Histology: {(rand_r2 - hist_r2).mean():.4f} mean R² drop")
print(f"  Random → Site: {(rand_r2 - site_r2).mean():.4f} mean R² drop")
print(f"\nDrugs with R²>0 by split:")
print(f"  Random: {(rand_r2 > 0).sum()} / {len(rand_r2)} ({100*(rand_r2 > 0).mean():.1f}%)")
print(f"  Histology: {(hist_r2 > 0).sum()} / {len(hist_r2)} ({100*(hist_r2 > 0).mean():.1f}%)")
print(f"  Site: {(site_r2 > 0).sum()} / {len(site_r2)} ({100*(site_r2 > 0).mean():.1f}%)")